In [108]:
from dotenv import load_dotenv, find_dotenv
from langchain_community.utilities import SQLDatabase

load_dotenv(find_dotenv())

True

In [109]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [122]:

examples = [
    {"input": "List all artists.", "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of invoices.",
        "query": "SELECT COUNT(*) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]

In [111]:
from langchain_community.vectorstores import Annoy
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Annoy,
    k=5,
    input_keys=["input"],
)

In [112]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [113]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [114]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")
artists[:50]

['Los Lonely Boys',
 'Santana Feat. The Project G&B',
 'Aquaman',
 'Sir Georg Solti & Wiener Philharmoniker',
 'Les Arts Florissants & William Christie',
 'Foo Fighters',
 'Santana Feat. Rob Thomas',
 'Zeca Pagodinho',
 'Banda Black Rio',
 'Santana Feat. Maná',
 'João Suplicy',
 'C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu',
 'Skank',
 'Lenny Kravitz',
 'Motörhead & Girlschool',
 'Matisyahu',
 'Nando Reis',
 'Corinne Bailey Rae',
 'Yo-Yo Ma',
 'Equale Brass Ensemble, John Eliot Gardiner & Munich Monteverdi Orchestra and Choir',
 'Queen',
 'João Gilberto',
 'Britten Sinfonia, Ivor Bolton & Lesley Garrett',
 'Gonzaguinha',
 'James Brown',
 'Os Mutantes',
 "Aerosmith & Sierra Leone's Refugee Allstars",
 'Ozzy Osbourne',
 'Xis',
 'Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart',
 'Itzhak Perlman',
 'Gene Krupa',
 'Planet Hemp',
 'Marcos Valle',
 'Ney Matogrosso',
 'Djavan',
 'R.E.M. Feat. Kate Pearson',
 'The Who',
 'Avril L

In [115]:
from langchain.agents.agent_toolkits import create_retriever_tool

vector_db = Annoy.from_texts(artists + albums, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [116]:
from langchain.agents import AgentType
from langchain_community.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [117]:
agent.invoke(
    "What was the total revenue from all sales?"
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `SELECT SUM(Total) AS TotalRevenue FROM Invoice`


[(2328.6,)]The total revenue from all sales is $2328.6.

> Finished chain.


{'input': 'What was the total revenue from all sales?',
 'output': 'The total revenue from all sales is $2328.6.'}

In [118]:
agent.invoke(
    "which album contributed the most to the total revenue?"
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `SELECT AlbumId, SUM(UnitPrice * Quantity) AS Revenue FROM InvoiceLine GROUP BY AlbumId ORDER BY Revenue DESC LIMIT 1`


Error: (sqlite3.OperationalError) no such column: AlbumId
[SQL: SELECT AlbumId, SUM(UnitPrice * Quantity) AS Revenue FROM InvoiceLine GROUP BY AlbumId ORDER BY Revenue DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Invoking: `sql_db_schema` with `InvoiceLine`



CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("InvoiceLineId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")
)

/*
3 rows from InvoiceLine table:
InvoiceLineId	InvoiceId	TrackId	UnitPrice	Quantity
1	1	2	0.99	1
2	1	4	0.99	1
3	2	6	0.99	1
*/
Invoking: `sql_db_query` with `SELECT T

{'input': 'which album contributed the most to the total revenue?',
 'output': 'The album that contributed the most to the total revenue is "Battlestar Galactica (Classic), Season 1".'}

In [119]:
agent.invoke(
    "Which country is most likely to listen to rock music?"
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `SELECT Country, COUNT(*) AS Total FROM Customer WHERE CustomerId IN (SELECT CustomerId FROM Invoice WHERE InvoiceId IN (SELECT InvoiceId FROM InvoiceLine WHERE TrackId IN (SELECT TrackId FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock')))) GROUP BY Country ORDER BY Total DESC LIMIT 1`


[('USA', 13)]The country most likely to listen to rock music is the USA.

> Finished chain.


{'input': 'Which country is most likely to listen to rock music?',
 'output': 'The country most likely to listen to rock music is the USA.'}

In [128]:
agent.invoke(
    "How many albums does blacksabbath have?"
)

# agent.invoke(
#     "How many albums does alis in chain have?"
# )



> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `SELECT COUNT(*) FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'Black Sabbath')`


[(2,)]Black Sabbath has 2 albums.

> Finished chain.


{'input': 'How many albums does blacksabbath have?',
 'output': 'Black Sabbath has 2 albums.'}

In [127]:
agent.invoke("How many albums where sold in the usa?")

# agent.invoke("How many albums where sold in USA?")



> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `SELECT COUNT(*) FROM Invoice WHERE BillingCountry = 'USA'`


[(91,)]There were 91 albums sold in the USA.

> Finished chain.


{'input': 'How many albums where sold in the usa?',
 'output': 'There were 91 albums sold in the USA.'}